# chunk to faiss with search

faiss db와 연결하여 인덱스 생성, 조회 및 데이터 조회를 수행해보는 예제입니다.

엘라스틱서치와의 결합을 통해 하이브리드 검색도 진행할 수 있습니다.

In [1]:
import os
from dotenv import load_dotenv

import joblib
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

import requests

c:\Users\jilp1\AppData\Local\pypoetry\Cache\virtualenvs\teddynote-parser2-2bulC10K-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

load_dotenv()
FAISS_API_KEY = os.getenv("API_KEY")
print(FAISS_API_KEY)

nlp13_access_token_IJkdk3d2


In [4]:
chunks = joblib.load('./data/final_documents.pkl')

In [5]:
# faiss 클라이언트 설정
session = requests.Session()
url = "http://3.34.62.202:8060/"

In [6]:
# 1. 텍스트 임베딩 모델 로드 (Hugging Face SentenceTransformer)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')  # 예

In [7]:
# 2. 텍스트 데이터를 벡터로 변환
text = "Faiss is a distributed, RESTful search engine."
vector = model.encode(text).tolist()  # 벡터를 리스트로 변환
print("vector", vector)
dimension = len(vector)
print("dimension:", dimension)

vector [-0.2365940362215042, -0.27777886390686035, -0.2769816219806671, -0.148578479886055, -0.044665779918432236, 0.18730643391609192, -0.2007625550031662, -0.10773374140262604, 0.017776891589164734, -0.2910635471343994, 0.04522012174129486, 0.24614793062210083, 0.22713202238082886, 0.3715372681617737, -0.23451651632785797, -0.08502563834190369, 0.2329074889421463, -0.13126130402088165, 0.28576672077178955, -0.1717987060546875, -0.10931780189275742, 0.0002679969184100628, 0.07361256331205368, -0.10275539755821228, -0.49946337938308716, -0.15136031806468964, -0.17468689382076263, 0.15553078055381775, 0.16283665597438812, -0.2169361561536789, 0.2636154890060425, -0.007321728393435478, -0.3107620179653168, 0.20946000516414642, -0.0660012885928154, -0.2510432004928589, -0.05435565114021301, 0.12379426509141922, -0.1986735761165619, 0.13712990283966064, -0.08966284990310669, 0.2245103418827057, -0.13555888831615448, -0.08458245545625687, -0.08591359108686447, -0.13387733697891235, 0.007383

In [9]:
# 3. 인덱스 생성
index_name = "prod-labq-documents-final-multi-minilm-l12-v2"
response = session.post(f"{url}/api/index", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "algorithm" : 1, "dimension": dimension}, timeout=15)
print(response.json())

{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-final-multi-minilm-l12-v2'}


In [10]:
# 4. 인덱스가 존재하는 지 확인
response = session.get(f"{url}/api/index", headers={"x-api-key" : FAISS_API_KEY}, params={"index" : index_name}, timeout=15)
print(response.json())

{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-final-multi-minilm-l12-v2'}


In [ ]:
# 5. 데이터 대량 삽입
for chunk in chunks:
    vectors = model.encode(chunk.page_content).tolist()
    response = session.post(f"{url}/api/context", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "input_vector" : vectors}, timeout=15)
    print(response.json())
    
    


In [20]:
print(len(chunks))

3629


In [18]:
# 6. 데이터 조회
query_text = "sk 하이닉스의 4q24 영업이익은?"
query_vector = model.encode(query_text).tolist()

response = session.post(f"{url}/api/context/search-by-vector", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "query_vector" : query_vector, "size" : 5}, timeout=15)
print(response.json())

{'status': 'ok', 'results': [{'document_id': 2066, 'distance': 6.5727219581604}, {'document_id': 181, 'distance': 6.588539123535156}, {'document_id': 3117, 'distance': 6.606011390686035}, {'document_id': 2070, 'distance': 6.61046028137207}, {'document_id': 1935, 'distance': 6.669358253479004}]}


In [19]:
# 8. 반환된 데이터를 디코딩
faiss_results = response.json()['results']
for item in faiss_results:
  print(item)
  print(chunks[item['document_id']])
  print("")

{'document_id': 2066, 'distance': 6.5727219581604}
page_content='20240920에 하나증권에서 발행한 롯데렌탈에 관한 레포트에서 나온 내용.

롯데렌탈은 쏘카 지분의 25.73%를 보유한 2대주주이다. SK는 기존 보유 쏘카 주식(총
17.9%)을 2차례에 걸쳐 롯데렌탈에 매각하기로 결정한 바 있고, 1차 지분 매각(지분율
기준 9.00%, 매매대금 660억원)은 2024년 2월 종결되었으며 2차 지분 매각(지분율 기
준 8.95%, 매매대금 660억원)이 중지 된 것이다. 쏘카의 실질 최대주주인 이재웅 대표
가 자기주식을 매입하며 현재 최대주주 및 특수관계인의 합산 지분율은 44.27%로, 롯데
렌탈과의 지분율 격차는 18.53%까지 벌어진 상태이다. SK의 나머지 지분(8.95%) 취득
여부와 상관 없이, 롯데렌탈이 쏘카의 최대주주가 되기는 어려운 상태라고 판단한다. 최
대주주(우호지분 포함) 지분율이 이미 44.27%로 너무 높기 때문이다. 설령 롯데렌탈의
2차 지분 매입이 진행되었다고 해도 여전히 최대주주와의 지분율 격차는 9.58%p 나 되
는 것이다.' metadata={'category': 'paragraph', 'coordinates': [{'x': 0.3294, 'y': 0.4417}, {'x': 0.9166, 'y': 0.4417}, {'x': 0.9166, 'y': 0.6438}, {'x': 0.3294, 'y': 0.6438}], 'page': 1, 'id': 7, 'company_name': '롯데렌탈', 'report_date': '20240920', 'securities_firm': '하나증권', 'source_file': '롯데렌탈_20240920_하나증권.pdf', 'content_types': {'markdown': '롯데렌탈은 쏘카 지분의 25.73%를 보유한 2대주주이다. SK는 기존 보유 쏘카 주식(총\n17.9%)을 2차례에 걸쳐 롯데렌탈에 매각하기로 결정한 바 있고, 

## Search With Elasticsearch

In [21]:
# Elasticsearch 클라이언트 설정
es = Elasticsearch("http://3.34.62.202:9200")

In [22]:
es_index_name = "prod-labq-documents-final-multi-minilm-l12-v2"

In [ ]:
response = es.indices.delete(index=es_index_name)
print(response)

In [24]:
# Elasticsearch에 인덱스 존재하는 지 확인

if not es.indices.exists(index=es_index_name):
    response = es.indices.create(index=es_index_name)
    print(response)
    print("Elasticsearch 인덱스 생성 완료")
    response = es.indices.refresh(index=es_index_name)
    print(response)
else:
    print("Elasticsearch 인덱스 이미 존재")
response = es.indices.stats(index=es_index_name)
print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'prod-labq-documents-final-multi-minilm-l12-v2'}
Elasticsearch 인덱스 생성 완료
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'docs': {'count': 0, 'deleted': 0, 'total_size_in_bytes': 0}, 'shard_stats': {'total_count': 1}, 'store': {'size_in_bytes': 227, 'total_data_set_size_in_bytes': 227, 'reserved_in_bytes': 0}, 'indexing': {'index_total': 0, 'index_time_in_millis': 0, 'index_current': 0, 'index_failed': 0, 'delete_total': 0, 'delete_time_in_millis': 0, 'delete_current': 0, 'noop_update_total': 0, 'is_throttled': False, 'throttle_time_in_millis': 0, 'write_load': 0.0}, 'get': {'total': 0, 'time_in_millis': 0, 'exists_total': 0, 'exists_time_in_millis': 0, 'missing_total': 0, 'missing_time_in_millis': 0, 'current': 0}, 'search': {'open_contexts': 0, 'query_total': 0, 'query_time_in_millis': 0, 'query_current': 0, 'query_failure': 0, 'fetch_tot

In [ ]:
# 데이터 대량 삽입

for chunk in chunks:
    text = chunk.page_content
    metadata = chunk.metadata
    vector = model.encode(text).tolist()  # 벡터를 리스트로 변환

    # Elasticsearch에 데이터 삽입
    doc = {
        "uid": metadata['uuid'],
        "nid": metadata['nid'],
        "text": text,
        "embedding": vector,
        "metadata": metadata
        
    }
    response = es.index(index=es_index_name, body=doc)
    print("Document indexed:", response)
    

In [27]:

query = {
    "query": {
        "match": {
            "text": query_text
        }
    }
}

# 검색 실행
response = es.search(index=es_index_name, body=query, size=5)

C:\Users\jilp1\AppData\Local\Temp\ipykernel_9568\2656234857.py:10: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=es_index_name, body=query, size=5)


In [28]:


print(response)
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")

{'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 173, 'relation': 'eq'}, 'max_score': 17.726849, 'hits': [{'_index': 'prod-labq-documents-final-multi-minilm-l12-v2', '_id': 'Lz12t5QBLfL4KSeUFqqo', '_score': 17.726849, '_ignored': ['metadata.content_types.html.keyword', 'text.keyword'], '_source': {'uid': 'be3f463c-419e-4f77-bbe9-a4592a763bc4', 'nid': 2354, 'text': '20241128에 SK증권에서 발행한 SK하이닉스에 관한 레포트에서 나온 내용.\n\nSK 증권은 4Q24 SK 하이닉스의 영업이익을 8.2 조원 (+17% QoQ)으로 예상한다.\n전통 B2C세트 수요 부진에 따른 Commodity 가격 하락에도 불구하고, HBM 등 고부\n가 중심의 확판과 레거시 출하 지양으로 우려보다 양호한 가격 (DRAM +10%,\nNAND -1%)을 예상하기 때문이다. DRAM 내 HBM 매출액 비중은 40% (+10%p\nQoQ)로 상승하며, 업황 우려가 본격화되는 시점에 높은 방어력을 확인할 것이다.', 'embedding': [-0.061736851930618286, 0.006447869818657637, -0.1095505952835083, -0.1444968730211258, -0.2072371393442154, -0.031334176659584045, -0.02756771817803383, 0.1741304248571396, -0.044077713042497635, -0.00780077651143074, 0.08699044585227

In [29]:



## Faiss에서 검색한 결과에서 document_id를 이용해서 Elasticsearch에서 데이터를 가져와 nid를 통해 디코딩.

nid_values = [item['document_id']+1 for item in faiss_results]

query = {
    "query": {
        "terms": {
            "nid": nid_values
        }
    }
}

response = es.search(index=es_index_name, body=query)
  


In [30]:
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")
    print(f"Metadata: {hit['_source']['metadata']}")
    print("")

ID: sz1wt5QBLfL4KSeULaFF, Score: 1.0
Content: {'uid': '14532049-359e-4274-9002-27120a5adbc6', 'nid': 182, 'text': '20241025에 DS증권에서 발행한 SK하이닉스에 관한 레포트에서 나온 내용.\n\nBrief Description: 표2 SK하이닉스 HBM 분기별 매출 추정\n\n|  | 1Q24 | 2Q24 | 3Q24P | 4Q24E | 1Q25E | 2Q25E | 3Q25E | 4Q25E | 2023 | 2024E | 2025E |\n| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |\n| DRAM Total Bit (1Gb equivalent, mn) | 16,339 | 19,982 | 19,481 | 20,455 | 19,432 | 20,404 | 22,036 | 22,256 | 64,839 | 76,256 | 84,127 |\n| HBM Bit 비중 (%) | 4 | 6 | 10 | 13 | 13 | 13 | 13 | 14 | 3 | 9 | 13 |\n| HBM Bit | 719 | 1,199 | 1,948 | 2,659 | 2,526 | 2,652 | 2,887 | 3,027 | 1,912 | 6,525 | 11,092 |\n| HBM ASP(U$/1Gb) | 1.29 | 1.39 | 1.50 | 1.58 | 1.58 | 1.58 | 1.64 | 1.72 | 1.12 | 1.49 | 1.63 |\n| 환율 | 1,350 | 1,360 | 1,345 | 1,345 | 1,345 | 1,345 | 1,345 | 1,345 | 1,322 | 1,350 | 1,345 |\n| HBM 매출액 (십억원) | 1,250 | 2,268 | 3,930 | 5,633 | 5,351 | 5,619 | 6,360 | 7,002 | 2,827 | 13,081 | 24,332 |\n| QoQ % | 6

In [31]:
## hybrid 검색 - Faiss와 Elasticsearch를 조합해서 검색 (Faiss로 검색한 결과 중에서 Elasticsearch로 bm25 검색)

query_text = "sk 하이닉스의 내년 주가는 오를 전망이야?"
query_vector = model.encode(query_text).tolist()

response = session.post(f"{url}/api/context/search-by-vector", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "query_vector" : query_vector, "size" : 100}, timeout=15)

faiss_results = response.json()['results']

In [ ]:

for item in faiss_results:
    print(item)
    doc = {
        "query": {
            "match": {
                "nid": item['document_id'] + 1
            }
        }
    }
    response = es.search(index=es_index_name, body=doc)
    print(response["hits"]["hits"][0]["_source"])
    print("")

In [33]:

nid_values = [item['document_id']+1 for item in faiss_results]

query = {
     "query": {
        "bool": {
            "filter": {
                "terms": {
                    "nid": nid_values
                }
            },
            "must": {
                "match": {
                    "text": query_text 
                }
            }
        }
    },
    "size": 10,  # 최상위 도큐먼트 10개
    "sort": [
        {
            "_score": {
                "order": "desc"
            }
        }
    ]
}

response = es.search(index=es_index_name, body=query)


In [34]:
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")
    print(f"Metadata: {hit['_source']['metadata']}")
    print("")

ID: fj14t5QBLfL4KSeUfK3L, Score: 9.587404
Content: {'uid': 'd4f45944-594b-4b0f-b98d-42941094a3ed', 'nid': 3201, 'text': '20241015에 SK증권에서 발행한 SK케미칼에 관한 레포트에서 나온 내용.\n\nSection Title: # 펀 더멘탈 대비 주가는 여전히 제자리\n\n4Q24 본업 이익 성장 이후 25 년 SK 케미칼의 본업은 추가적인 개선이 가능할 것으\n로 예상된다. 1H25 SK 케미칼은 기존 설비개조를 통해 고부가 Copolyester 생산능력\n을 확대할 계획이다. 설비 개조과정에서 일시적 가동률 하락이 예상되나, 2H25 부터 본\n격 가동된다면 수익성의 추가적인 개선이 가능할 것으로 예상된다. 단기적으로 CR(화\n학적 재활용) 소재의 실적 가시화는 제한적일 전망이지만, 지속적으로 고객사를 확보해\n나가고 있는 점도 성장 포인트로서 유효하다. 투자의견 ‘매수’ 유지한다.', 'embedding': [-0.07754214853048325, 0.1527159959077835, -0.15567445755004883, -0.06351787596940994, -0.026207029819488525, -0.03956063836812973, -0.15313909947872162, 0.12547127902507782, -0.1384616494178772, -0.14231635630130768, 0.19660478830337524, 0.0900459811091423, -0.002111372072249651, -0.0869080051779747, 0.009655115194618702, -0.184348464012146, -0.08043620735406876, 0.03216024860739708, -0.03779175132513046, -0.01787550374865532, -0.09093207120895386, -0.07844585925340652, 0.1722

In [37]:
for hit in response["hits"]["hits"]:
  if '3Q24' in hit['_source'] :
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")
    print(f"Metadata: {hit['_source']['metadata']}")
    print("")
    